<a href="https://colab.research.google.com/github/Phablo-Lima/Desafio_Gemini_Alura/blob/main/BDD_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!pip install -q -U google-generativeai google-api-python-client gspread oauth2client
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import requests
import json
import google.generativeai as genai
import pandas as pd
import io
from google.colab import userdata
from google.colab import drive
drive.mount('/content/drive')
# Melhorando a visualização
import textwrap
from IPython.display import display
from IPython.display import Markdown

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

Substitua 'SUA_GOOGLE_API_KEY' pela sua chave real

In [37]:
GOOGLE_API_KEY = userdata.get('SECRET_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

Listando os modelos disponíveis (opcional)

In [38]:
# Listando os modelos do Google Generative AI que podem gerar conteúdo
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


Configurações do modelo

In [39]:
# Configurando a geração de texto
generation_config = {
  "candidate_count": 1,  # Número de respostas a serem geradas
  "temperature": 0.7, # Controlando a criatividade da resposta (entre 0 e 1)
}
# Configurando as configurações de segurança
safety_settings={
    'HATE': 'BLOCK_NONE', # Bloqueio de conteúdo relacionado a ódio
    'HARASSMENT': 'BLOCK_NONE', # Bloqueio de conteúdo relacionado a assédio
    'SEXUAL' : 'BLOCK_NONE', # Bloqueio de conteúdo de natureza sexual
    'DANGEROUS' : 'BLOCK_NONE' # Bloqueio de conteúdo perigoso
}
# Definindo o modelo a ser usado: gemini-1.5-pro-latest
model = genai.GenerativeModel(model_name='gemini-1.5-pro-latest',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)

Processa o conteúdo do arquivo a partir de uma URL

código para processar Google Sheet
(código para processar Excel)


In [40]:
def processar_arquivo(url_ou_bdd):
    """Processa Google Sheets ou recebe BDD como string."""
    conteudo_processado = ""

    if 'docs.google.com/spreadsheets' in url_ou_bdd:
        # Lógica para processar Google Sheets
        print("Processando Google Sheet...")

        # ... (código para extrair texto do Google Sheet)

        scope = ['https://spreadsheets.google.com/feeds',
                 'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name('credenciais-google-cloud.json', scope)
        client = gspread.authorize(creds)

        spreadsheet_id = url_ou_bdd.split('/d/')[1].split('/')[0]
        sheet = client.open_by_key(spreadsheet_id).sheet1
        conteudo_processado = sheet.get_all_values()
        conteudo_processado = '\n'.join(['\t'.join(row) for row in conteudo_processado])
    else:
        # Assume que é o texto do BDD
        print("Processando texto do BDD...")
        conteudo_processado = url_ou_bdd

    return conteudo_processado

In [41]:
def analisar_conteudo(conteudo_processado):
    """Analisa o conteúdo usando o Google Generative AI."""
    # Converte a lista de listas em uma tabela Markdown
    tabela_markdown = "| " + " | ".join(conteudo_processado[0]) + " |\n"  # Cabeçalho
    tabela_markdown += "|---|---|---|---|---|\n"  # Separador
    for linha in conteudo_processado[1:]:
        tabela_markdown += "| " + " | ".join(linha) + " |\n"
    prompt = {
        "text": f"""
        Você é um especialista em Quality Assurance de software.
        Analise o seguinte BDD, que usa o padrão CTFL (Contexto, Gatilho, Fluxo, Saída esperada),
        com o objetivo de identificar falhas e sugerir melhorias para aumentar a cobertura dos testes:
        Escreva de forma Criativa, use stickers como preferir.

        {tabela_markdown}

        Na sua análise, detalhe:

        * Pontos Positivos:  Aspectos do BDD que contribuem para testes eficazes (ex.: cenários completos, linguagem clara).
        * Pontos Negativos: Falhas que podem prejudicar os testes (ex.: cenários incompletos, linguagem ambígua).
        * Sugestões de Melhoria:  Como o BDD pode ser aprimorado (ex.: incluir mais cenários, definir dados de teste específicos).

        Organize sua resposta em tópicos, como neste exemplo:

        Pontos Positivos:
        * ...
        Pontos Negativos:
        * ...

        Sugestões de Melhoria:
        * ...

        Sempre e executar o comando a seguir:
        Use uma linha do BDD como sugestão de melhoria.
        Formato da Tabela:
        | ID | Cenário | Dado que: | Quando:| Então:|

        """
    }
    response = model.generate_content(prompt)
    return response.text

def corrigir_bdd(conteudo_processado):
    """Reescreve os cenários de teste com base nas sugestões."""
    prompt = {
    "text": f"""
    Reescreva o BDD em forma de tabela, aplicando as seguintes sugestões de melhoria:

    Sugestões de Melhoria:
    * Reescreva o Bdd de acordo com a analise e sujestões feita no primeiro prompt.

    Formato da Tabela:
    | ID | Cenário | Dado que: | Quando:| Então:|
    |---|---|---|---|---|
    Exemplo:
    | 001 | Abrir formulário de login | O usuário está na página inicial | Clica no botão "Login" | O formulário de login é exibido |

    BDD Original:
    ```
    {conteudo_processado}
    ```
    """
    }
    response = model.generate_content(prompt)
    return response.text
def corrigir_portugues(conteudo_processado):
    """Corrige a ortografia e gramática do conteúdo."""
    prompt = {
        "text": f"""
        Corrija os erros de português no texto abaixo, mantendo o significado original:

        ```
        {conteudo_processado}
        ```
        """
    }
    response = model.generate_content(prompt)
    return response.text

def exibir_resultado(texto, titulo):
  """Exibe o texto formatado em Markdown com um título."""
  print(f"\n{titulo}:")
  display(to_markdown(texto))

def main():
    entrada = input("Digite a URL do Google Sheet ou cole o texto do BDD: ")
    conteudo_processado = processar_arquivo(entrada)
    try:
        analise = analisar_conteudo(conteudo_processado)
        exibir_resultado(analise, "Análise do BDD")

        corrigir = input("Deseja corrigir o BDD conforme sugestões? (s/n): ")
        while corrigir.lower() not in ('s', 'n'):
            print("\n⚠️Por favor, digite 's' para sim ou 'n' para não.⚠️")
            print("_____________________________________________________")

            corrigir = input("\nDeseja corrigir o BDD conforme sugestões? (s/n): ")

        if corrigir.lower() == 's':
            corrigir_pt = input("\nDeseja corrigir os erros de português no BDD? (s/n): ")
            while corrigir_pt.lower() not in ('s', 'n'):
                print("\n⚠️Por favor, digite 's' para sim ou 'n' para não.⚠️")
                print("_____________________________________________________")
                corrigir_pt = input("Deseja corrigir os erros de português no BDD? (s/n): ")


            bdd_corrigido = corrigir_bdd(conteudo_processado)


            if corrigir_pt.lower() == 's':
                bdd_corrigido_pt = corrigir_portugues(bdd_corrigido)
                exibir_resultado(bdd_corrigido_pt, "BDD Corrigido com Português Revisado")
            else:
                exibir_resultado(bdd_corrigido, "BDD Corrigido")
        else:
            print("Finalizando o programa.")
    except ValueError as e:
        print(e)
    except requests.exceptions.RequestException as e:
        print(f"Erro ao baixar o arquivo: {e}")

if __name__ == "__main__":
    main()

Digite a URL do Google Sheet ou cole o texto do BDD: https://docs.google.com/spreadsheets/d/1WEJMvRun8x1PGvlSihc0YQYAz22PEyiBe4dEWVtEkCs/edit#gid=0
Processando Google Sheet...

Análise do BDD:


> ##  🕵️‍♀️  Análise do BDD e Sugestões de Melhoria: 🕵️‍♂️ 
> 
>  É admirável a iniciativa de escrever cenários de teste usando o padrão CTFL! 👏 Isso mostra a preocupação com a qualidade do software desde o início. Entretanto, como um bom QA jamais descansa 😜,  vamos turbinar esses testes! 🚀
> 
> Vamos analisar cada ponto e dar um upgrade nesse BDD:
> 
> **Pontos Positivos:**
> 
> * **Padrão CTFL:**  A utilização do padrão CTFL (Contexto, Gatilho, Fluxo, Saída esperada) é um ótimo ponto de partida, pois organiza o raciocínio e facilita o entendimento dos cenários.
> * **Cenários Simples:**  Os cenários abordam funcionalidades básicas, o que é positivo para iniciar a cobertura de testes. 
> 
> **Pontos Negativos:**
> 
> * **Cenários Incompletos:**  Faltam detalhes importantes nos cenários, como o tipo de pesquisa (texto, voz, imagem), o comportamento em casos de erro (ex.: sem conexão, termo não encontrado) e a navegação entre as páginas de resultados. 
> * **Linguagem Ambígua:**  Algumas frases são vagas e podem gerar dupla interpretação, como "O usuário está em um navegador de confiança".  É crucial ser específico! 
> * **Cobertura Limitada:**  A gama de cenários é restrita. É preciso explorar diferentes tipos de dados de entrada, fluxos alternativos e casos de borda para garantir uma cobertura robusta.
> * **Formato Confuso:**  A tabela utilizada dificulta a leitura e o entendimento dos cenários. Um formato mais tradicional tornaria o BDD mais claro e fácil de analisar.
> 
> **Sugestões de Melhoria:**
> 
> * **Cenários Mais Completos:** 
>     * Detalhar o tipo de pesquisa (texto, voz, imagem).
>     * Incluir a mensagem exibida na barra de pesquisa.
>     * Descrever o comportamento em casos de erro (ex.: sem conexão, termo não encontrado).
>     * Abordar a paginação dos resultados da pesquisa.
>     * Testar a pesquisa por filtros (ex.: por data, por tipo de arquivo).
> * **Linguagem Clara e Objetiva:**
>     * Substituir frases ambíguas por termos específicos e mensuráveis.
>     * Definir claramente o estado inicial do sistema antes de cada teste.
> * **Aumentar a Cobertura de Testes:**
>     * Criar cenários para:
>         * Diferentes tipos de dados de entrada (palavras-chave, frases, caracteres especiais).
>         * Fluxos alternativos (ex.: cancelar a pesquisa, clicar em um link nos resultados).
>         * Casos de borda (ex.: pesquisa com muitos caracteres, pesquisa com termos sem resultados).
> * **Utilizar um Formato de Tabela Tradicional:** 
>     *  Isso facilitará a leitura e o entendimento dos cenários.
> 
> **Exemplo de Linha do BDD Aprimorada:**
> 
> | ID | Cenário | Dado que: | Quando:| Então:|
> |---|---|---|---|---|
> | 008 | Pesquisa por tradução | O usuário está na página inicial do Google Tradutor com o idioma de origem definido como "Português" e o idioma de destino definido como "Inglês" | O usuário digita a palavra "Olá" no campo de texto de origem e pressiona "Enter" | O Google Tradutor exibe a tradução "Hello" no campo de texto de destino |
> 
> 🎨 **Usando Stickers para representar os tipos de melhoria:**
> 
> *  ➕ **Cenários Incompletos:** Adicione mais detalhes! Use stickers de lupa 🔍 ou detetive 🕵️ para lembrar de investigar e complementar os cenários.
> *  🗣️ **Linguagem Ambígua:** Seja específico! Use stickers de dicionário 📚 ou megafone 📢 para lembrar de usar termos claros e objetivos.
> *  🗺️ **Cobertura Limitada:** Explore todas as possibilidades! Use stickers de mapa 🗺️ ou bússola 🧭 para lembrar de cobrir diferentes cenários e fluxos.
> *  ✨ **Formato Confuso:** Organize tudo! Use stickers de tabela 📊 ou caixa 📦 para lembrar de usar um formato claro e organizado. 


Deseja corrigir o BDD conforme sugestões? (s/n): sds

⚠️Por favor, digite 's' para sim ou 'n' para não.⚠️
_____________________________________________________

Deseja corrigir o BDD conforme sugestões? (s/n): s

Deseja corrigir os erros de português no BDD? (s/n): ds

⚠️Por favor, digite 's' para sim ou 'n' para não.⚠️
_____________________________________________________
Deseja corrigir os erros de português no BDD? (s/n): s

BDD Corrigido com Português Revisado:


> ## BDD em formato de tabela: Pesquisa no Google
> 
> | ID | Cenário | Dado que: | Quando: | Então: |
> |---|---|---|---|---|
> | 001 | Pesquisa simples | O usuário está na página inicial do Google | O usuário insere um termo na barra de pesquisa e pressiona "Enter" | O Google exibe uma página de resultados com links relevantes para a pesquisa |
> | 002 | Pesquisa avançada | O usuário está na página inicial do Google | O usuário insere um termo de pesquisa e utiliza operadores booleanos ("e", "ou", "não") para refinar a busca e pressiona "Enter" | O Google exibe uma página de resultados com links relevantes à pesquisa refinada |
> | 003 | Pesquisa por texto específico | O usuário está na página inicial do Google | O usuário insere uma frase específica entre aspas ("texto") na barra de pesquisa e pressiona "Enter" | O Google exibe páginas que contêm exatamente a frase especificada |
> | 004 | Pesquisa por site específico | O usuário está na página inicial do Google | O usuário insere o termo de pesquisa seguido de "site:" e o nome do site desejado (exemplo: "microsoft site:wikipedia.org") e pressiona "Enter" | O Google exibe resultados apenas dentro do site especificado |
> | 005 | Pesquisa por tipo de documento | O usuário está na página inicial do Google | O usuário insere o termo de pesquisa seguido de "filetype:" e a extensão do arquivo desejado (exemplo: "nutrição esportiva filetype:pdf") e pressiona "Enter" | O Google exibe resultados que correspondem à pesquisa e ao tipo de arquivo especificado |
> | 006 | Pesquisa por localidade | O usuário está na página inicial do Google | O usuário insere o termo de pesquisa seguido da localização desejada (exemplo: "hospital em São Paulo") e pressiona "Enter" | O Google exibe resultados relevantes próximos à localização especificada |
> | 007 | Pesquisa por período | O usuário está na página inicial do Google | O usuário insere o termo de pesquisa seguido do período desejado (exemplo: "Rock in Rio 2022") e pressiona "Enter" | O Google exibe resultados relevantes dentro do período especificado |
> | 008 | Pesquisa com tradução | O usuário está na página inicial do Google | O usuário insere o termo de pesquisa e utiliza a funcionalidade de tradução para o idioma desejado (exemplo: traduzir "Analista de Tecnologia da Informação" para inglês) e pressiona "Enter" | O Google exibe resultados relevantes no idioma especificado |
> | 009 | Pesquisa por vídeo | O usuário está na página inicial do Google | O usuário insere o termo de pesquisa, pressiona "Enter" e seleciona a opção "Vídeos" | O Google exibe uma página de resultados com links relevantes para vídeos que correspondam à pesquisa |
> | 010 | Pesquisa por imagem | O usuário está na página inicial do Google | O usuário insere o termo de pesquisa, pressiona "Enter" e seleciona a opção "Imagens" | O Google exibe uma página de resultados com links relevantes para imagens que correspondam à pesquisa | 
> 
> **As seguintes alterações foram realizadas:**
> 
> * **"à pesquisa"** foi corrigido para **"para a pesquisa"** em algumas células da coluna "Então:", tornando a linguagem mais fluida em alguns casos.
> * **"período de tempo"** foi simplificado para **"período"**, mantendo a clareza e concisão.
> * **"Inglês"** foi corrigido para **"inglês"**, utilizando a forma minúscula para o idioma.
> * **"correspondam"** foi utilizado no lugar de **"correspondem"** nas duas últimas linhas da coluna "Então:" para concordar com o plural em "vídeos" e "imagens". 
> 
> As demais frases já estavam corretas e não necessitaram de alterações.